In [1]:
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt

## opening the file
hdul = fits.open("BGS_BRIGHT_NGC_clustering.dat.fits", memmap = True) ## use memmap as the file is big 
hdul.info()

## stuff in the file
data = hdul[1].data 
## convert the data to a table (could use the .data thingy but making a table makes it easier)
data_table = Table(data)
hdul.close()

Filename: BGS_BRIGHT_NGC_clustering.dat.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  LSS           1 BinTableHDU     55   2909876R x 18C   [K, D, K, D, D, 1A, D, D, D, D, D, E, E, E, E, E, D, D]   


In [2]:
from astropy.coordinates import SkyCoord, search_around_sky, Angle
from astropy.cosmology import Planck18
import astropy.units as u
import numpy as np

## identifying possible candidates:

## First trying to identify galaxies within a specified cone
## radius on either side of candidate galaxy
mpc_rad = 1 * u.Mpc
part_table = data_table[1:25000]
z = part_table['Z']

# coordinates of each galaxy using ra and dec
ra = part_table['RA']
dec = part_table['DEC']
coords = SkyCoord(ra=ra*u.deg, dec=dec*u.deg) ## coordinates of each galaxy in the data set
d_a = Planck18.angular_diameter_distance(z) ## angular diameter distance

theta = Angle((mpc_rad / d_a).value, unit=u.rad) ## defining the angular radius of the cone

theta_max = np.max(theta) ## maximum theta from array as search_around_sky cannot take an array for seplimit

cand, neigh, sep, _ = search_around_sky(coords, coords, seplimit = theta_max) ## searching for pairs of galaxies within a specific separation limit

theta_cut = theta[cand] > sep.to(u.rad)
## to make a cut on the redshifts (neighbours must be +/- 0.01 the redshift of the candidates)
delta_z = np.abs(z[cand]-z[neigh]) <= 0.01

In [3]:
## determining luminosity of each galaxy
flux_rbandnmgy = part_table['flux_r_dered']
app_mags = (22.5 - 2.5 * np.log10(flux_rbandnmgy.value)) * u.mag
lum_dists = Planck18.luminosity_distance(z)
abs_mags = app_mags.value - (5*np.log10(lum_dists.value/0.00001))
magsun_r = 4.64
luminosities = 10**(-0.4 * (abs_mags - magsun_r))

neigh_lum = luminosities[neigh]
cand_lum = luminosities[cand]

lum_cut = neigh_lum/cand_lum > 0.1 ## keep neighbours that are more than 0.1 luminosity of the candidate 

mag_cut = abs_mags[cand] < -17 ## keep candidate galaxies that have a magnitude less than -17

In [4]:
self_cut = cand != neigh
unique_cut = cand > neigh

cut = theta_cut & delta_z & lum_cut & mag_cut & self_cut & unique_cut
## cutting a cone around each independent candidate galaxy and ensures that the cone is larger than separation b/w candidate and neighbour

cand, neigh, sep = cand[cut], neigh[cut], sep[cut] ## applying the cuts

print(f'Candidate galaxies: {cand}')
print(f'Neighbouring galaxies: {neigh}')

Candidate galaxies: [    1     2     2 ... 24986 24987 24994]
Neighbouring galaxies: [    0     0     1 ... 24981 24983 24992]
